# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/1f5yE/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298457577067                   -0.85    5.0         
  2   -8.300205506546       -2.76       -1.26    1.2    110ms
  3   -8.300435488417       -3.64       -1.89    2.8    132ms
  4   -8.300462068870       -4.58       -2.74    2.5    130ms
  5   -8.300464547430       -5.61       -3.23    2.8    165ms
  6   -8.300464612408       -7.19       -3.39    4.8    201ms
  7   -8.300464628511       -7.79       -3.54    8.1    239ms
  8   -8.300464636035       -8.12       -3.67    1.5    125ms
  9   -8.300464641224       -8.28       -3.85    1.8    122ms
 10   -8.300464642752       -8.82       -3.98    1.1    115ms
 11   -8.300464643849       -8.96       -4.30    1.9    173ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67386357715                   -0.70    6.0         
  2   -16.67795155775       -2.39       -1.14    1.0    304ms
  3   -16.67913170914       -2.93       -1.87    3.8    358ms
  4   -16.67923820894       -3.97       -2.63    3.5    350ms
  5   -16.67928532300       -4.33       -3.10    4.5    440ms
  6   -16.67928619143       -6.06       -3.48    2.5    309ms
  7   -16.67928621377       -7.65       -3.99    2.6    328ms
  8   -16.67928622142       -8.12       -4.60    2.8    329ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32602588141                   -0.56    7.2         
  2   -33.33423544855       -2.09       -1.00    1.2    1.07s
  3   -33.33585499199       -2.79       -1.73    6.2    1.50s
  4   -33.33611486020       -3.59       -2.63    5.8    1.36s
  5   -33.33675829348       -3.19       -2.20    7.0    1.80s
  6   -33.33679829024       -4.40       -2.25    1.0    976ms
  7   -33.33694261140       -3.84       -3.33    1.8    1.16s
  8   -33.33694343970       -6.08       -3.54    3.4    1.25s
  9   -33.33694367301       -6.63       -3.79    1.9    1.05s
 10   -33.33694378043       -6.97       -4.76    2.4    1.18s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298352643106                   -0.85    5.4         
  2   -8.300247967380       -2.72       -1.59    1.0   83.9ms
  3   -8.300433497787       -3.73       -2.68    3.9    126ms
  4   -8.300443441720       -5.00       -2.58    4.0    174ms
  5   -8.300464393318       -4.68       -3.33    1.0   89.8ms
  6   -8.300464607724       -6.67       -3.90    4.9    167ms
  7   -8.300464641863       -7.47       -4.40    4.1    166ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32537379904                   -0.56    6.5         
  2   -33.29515765243   +   -1.52       -1.26    1.0    850ms
  3   +3.111479075028   +    1.56       -0.29    6.6    2.00s
  4   -33.32167205914        1.56       -1.65    6.4    1.85s
  5   -33.25041370163   +   -1.15       -1.33    3.2    1.43s
  6   -33.16189338018   +   -1.05       -1.44    3.2    1.33s
  7   -33.16888493383       -2.16       -1.45    4.2    1.32s
  8   -33.33636036875       -0.78       -2.30    3.6    1.21s
  9   -33.33665889493       -3.53       -2.61    3.6    1.34s
 10   -33.33669703201       -4.42       -2.59    2.2    1.19s
 11   -33.33691770498       -3.66       -3.07    1.8    1.01s
 12   -33.33692885088       -4.95      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.